In [100]:
import midi_info
import numpy as np
import pandas as pd
import torch
from copy import deepcopy
from typing import NewType, List, Dict
import re

_Types_

In [22]:
Note = NewType('Note', [str, int]) # Change the list to a "or" (?)
Tensor = torch.Tensor

One-Hot Encoding of the identifiers

In [119]:
N_notes = 132 - 11 # Fix this info inside the .py file

def note_to_vector(note: Note) -> Tensor:
    if type(note) == str:
        assert note in set(midi_info.Notes), 'Invalid Note'
        note = midi_info.notes_to_id[note]
    elif type(note) == int:
        assert note > 10 and  note < 133, 'Invalid Note'
        
    vector = torch.zeros(N_notes)
    vector[midi_info.id_to_vecIndex(note)] = 1
    
    return vector.reshape(1, -1)

def sequence_to_tensor(sequence: List[str]) -> Tensor:
    vector_list = []
    for note in sequence:
        vector_list.append(note_to_vector(note))
    
    return torch.cat(vector_list, dim = 0)

__Jazz ML ready MIDI__

https://www.kaggle.com/saikayala/jazz-ml-ready-midi

In [91]:
raw_data = pd.read_csv('/home/roboto/Documents/GitHub/tutorials/data/Jazz_MIDI_ML_Ready/Jazz-midi.csv', index_col = 'Name').drop('Unnamed: 0', axis = 1)
raw_data

,Notes,Len_Sequence,Unique_notes,len_Uni_Notes
Name,,,,
BreezeAndI.mid,"['E-2', 'B-2', 'B-2', 'E-3', 'B-2', 'B-2', 'E-...",427,"{'C3', 'F1', 'A2', 'B-0', 'E-1', 'E3', 'B1', '...",27
IfIHadYou.mid,"['10.2.5', '10.2.5', '2.5.8', '2.5.8', '0.3.7'...",204,"{'7.10.2', '6.9.0.2', '9.10.0.2.4', '6.9.0', '...",37
IllBeSeeingYou.mid,"['E-2', 'E-2', 'B-2', 'B-2', 'G2', 'D3', 'F2',...",455,"{'C3', 'F1', 'B-0', 'A2', 'E-1', 'D3', 'B1', '...",29
JustAGame.mid,"['B3', '6.11', 'B1', 'E4', 'B1', 'F#4', 'B1', ...",1572,"{'C3', '9.2', 'A2', 'B4', 'E3', 'A3', 'D5', 'B...",39
Unforgettable.mid,"['G2', 'G1', 'G2', 'G1', 'G2', 'G2', 'G2', 'C#...",421,"{'A0', 'C3', 'F1', 'A2', 'E-1', 'E3', 'G1', 'D...",28
...,...,...,...,...
ThatsAllGreatAmericanSongbook.mid,"['B-1', 'B-1', 'B-1', 'B-1', 'B-1', 'B-1', 'B-...",280,"{'G1', 'F#2', 'G2', 'E-2', 'F1', 'C#2', 'A2', ...",13
Respect.mid,"['C2', 'C2', 'A1', 'G1', 'C2', 'C2', 'G1', 'F1...",437,"{'G1', 'B1', 'F#2', 'E2', 'D2', 'G2', 'E-2', '...",17
ATimeForUs.mid,"['D3', '2.5.9', '2.5.9', 'E-3', '3.7.10', '3.7...",193,"{'D3', '7.10.2', '8.0.3', 'G#2', '2.5.9', 'G2'...",15


Extracting the sequence of notes from each music.

In [93]:
note_regex = re.compile(r'\w[b#]?\d')

def get_note_sequence(raw_data: str) -> List[str]:
    return note_regex.findall(raw_data)

def get_music_dictionary(data: pd.DataFrame) -> Dict:
    musics = {}
    
    for music in data.index:
        note_sequence = get_note_sequence(data['Notes'][music].replace('-', 'b'))
        if len(note_sequence) != 0:
            musics[music.replace('.mid', '')] = note_sequence
            
    return musics

In [101]:
data = get_music_dictionary(raw_data)
sample = deepcopy(data['BreezeAndI'])

Transforming each sequence of notes into a tensor

(sequence_length, number_of_features)

In [133]:
sequence_to_tensor(sample).shape

torch.Size([427, 121])